SPDX-FileCopyrightText: Copyright (c) 2022 NVIDIA CORPORATION & AFFILIATES. All rights reserved.

SPDX-License-Identifier: MIT

# Demo: Triton Model Analyzer

## Assumptions
You have an access to host, where you can run docker containers. The host is connected to the Internet.

## Prepare your model repository


For the next commands to work, it's important to clone this repository to the filesystem, which supports symbollic links, transparently for docker. Any linux disk partition is sufficient. NFS and NTFS are not.

As an example, we're using [Hi-Fi GAN](https://ngc.nvidia.com/catalog/models/nvidia:nemo:tts_hifigan) model from [NeMo](https://github.com/NVIDIA/NeMo).
To get it's ONNX one should run the NeMo container in the current directory.
```
docker run --rm --gpus '"device=0"' -it --ipc=host \
-v $HOME/:/ext_home \
-v ${PWD}:${PWD} \
-w ${PWD} \
--name ${USER}_nemo \
nvcr.io/nvidia/nemo:1.3.0 \
-- python get_hifigan.py
```

The command we run inside the container
```
python get_hifigan.py
```
is equivalent to the next cell:

In [ ]:
from nemo.collections.tts.models import HifiGanModel

model = HifiGanModel.from_pretrained(model_name="tts_hifigan")
model.export("./hifigan.onnx")

model = HifiGanModel.from_pretrained(model_name="tts_hifigan")
model.export("./hifigan.pt")

After running the cell above, two files are to appear in the current directory: `hifigan.onnx` and `hifigan.pt`

We'll need the ONNX model to experiment with Model Analyzer — this is the tool, that helps select the optimal inference config, within the specific backrnd. We need to copy  `hifigan.onnx` to `model_repository/hifigan/1/model.onnx`

```
mkdir -p model_repository/hifigan/1
cp hifigan.onnx model_repository/hifigan/1/model.onnx
```

TorchScript will be required later, for the Model Navigator experiments. It help in selecting the most optimal backend fot the specific model.
Having these files, the NeMo container can be stopped. For this, it's sufficient to exit its shell due to the `--rm` flag


## Curl and Perf Analyzer

A quick Triton test launch:
```
docker run --rm --gpus '"device=0"' -it --ipc=host \
-v $HOME/:/ext_home \
-v ${PWD}:${PWD} \
-w ${PWD} \
-p 8000:8000 \
-p 8001:8001 \
-p 8002:8002 \
--name ${USER}_triton \
nvcr.io/nvidia/tritonserver:22.12-py3 \
tritonserver --model-repository ${PWD}/model_repository --log-verbose 4
```

Note, that such amount of logs can negatively affect the performance, and is recommended for debug only.

After this, one can run in another terminal
```
curl -kv -X POST 'http://127.0.0.1:8000/v2/models/hifigan/infer' \
 -H 'accept: application/json' \
 -H 'Content-Type: application/octet-stream' \
 -H 'connection: keep-alive' \
 -d @hifigan_curl_data.json
```

Note, that it uses an HTTP protocol, which has quite high overhead.



No one can launch Triton SDK container in yet another terminal:
```
docker run --rm --gpus '"device=0"' -it --ipc=host \
-v $HOME/:/ext_home \
-v ${PWD}:${PWD} \
-w ${PWD} \
--net=host \
--name ${USER}_triton_sdk \
nvcr.io/nvidia/tritonserver:22.12-py3-sdk \
/bin/bash
```

One can save perf_analyzer help for later usage:
```
perf_analyzer --help 2>&1 | tee perf_analyzer_help.txt
```
And then measure the model performance

```
perf_analyzer -m hifigan --shape "spec:80,140"
```

The command above is again using the inefficient HTTP. The optimal launch will use GRPC, shared memory, batch size != 1 and several streams:

```
perf_analyzer -m hifigan --shape "spec:80,140" \
-b 4 \
-i gRPC \
--concurrency-range 1:3 \
--shared-memory "cuda" \
--output-shared-memory-size 60000000
```

On a V100 the performance gain is 30%. But which set of hyperparameters is the most optimal? Model Analyzer to the rescue!



## Model Analyzer Launch

Model Analyzer is used to select the optimal model cofig both for offline and online modes. To do it, it creates models with various configurations, launches the Triton container and uses Perf Analyzer to measure the performance.

Model Analyzer предназначен для выбора оптимальной конфигурации моделей для онлайн или оффлайн режима. Для этого он создаёт модели с различными конфигурациями, самостоятельно запускает контейнер с Triton'ом и использует Perf Analyzer для снятия метрик. Model Analyzer [Open Source](https://github.com/triton-inference-server/model_analyzer) и написан на Python.

Этот ноутбук рекомендуется исполнять в последнем контейнере с Triton SDK. При этом нужно подмонтировать полный путь до этого ноутбука под таким же путём, как и снаружи, чтобы model_analyzer мог удобно подмонтировать его к контейнеру с Triton'ом. Если запускать контейнер в папке с этим ноутбуком, рекомендуемая команда запуска

```
docker run --rm --gpus '"device=0"' -it --ipc=host \
-v $HOME/:/ext_home \
-v /var/run/docker.sock:/var/run/docker.sock \
-v ${PWD}:${PWD} \
-w ${PWD} \
--net=host \
--name ${USER}_triton_sdk \
nvcr.io/nvidia/tritonserver:21.11-py3-sdk \
/bin/bash
```

Для запуска ноутбука в контейнере можно дополнительно установить ipykernel, а можно просто копировать все последующие команды в терминал.
```
pip install ipykernel
```

Обратите внимание на подмонтированный docker.sock, который позволяет запускать контейнеры.

Если доступ на машину с GPU для инференса есть только через Kubernetes, это [тоже поддерживается](https://github.com/triton-inference-server/model_analyzer/blob/main/docs/kubernetes_deploy.md), но я не буду на этом останавливаться.

Перед запуском команд на тестирование следует убедиться, что на машине сейчас нет запущенного контейнера с name=tritonserver. Следующая команда остановит и удалит такой контейнер, если он был.
```
docker rm -f tritonserver 
```


## Model Analyzer Config


Из моделей, с которыми будет работать model analyzer, необходимо создать стандартный репозиторий моделей для Triton. У меня в репозитории в папке `model_repository` одна модель hifigan в одной версии: 1. 

Есть два режима подбора параметров Model Analyzer: 
[автоматический](https://github.com/triton-inference-server/model_analyzer/blob/main/docs/config_search.md#automatic-configuration-search)
и [ручной](https://github.com/triton-inference-server/model_analyzer/blob/main/docs/config_search.md#Manual-Configuration-Search)
— [введение в команду model-analyzer profile](https://github.com/triton-inference-server/model_analyzer/blob/main/docs/cli.md#subcommand-profile)

Я подготовил для Hi-Fi GAN два конфига: ручной [profile_config_manual.yaml](profile_config_manual.yaml) и
автоматический [profile_config_auto.yaml](profile_config_auto.yaml). Они отличаются значением параметра `run_config_search_disable`. Больше подробностей о других параметрах [тут](https://github.com/triton-inference-server/model_analyzer/blob/main/docs/config.md#configuring-model-analyzer).

**В ручном конфиге**  Измеряются задержки и пропускная способность Hi-FI GAN на различных парметрах размера батча и одновременных запросов, и при различном количестве инстансов.

**В автоматическом конфиге** эти же параметры, а так же наличие динамического батчинга, подбираются в автоматическом режиме.

**В обоих конфигах необходимо задать полный путь до выходного репозитория моделей**, в котором можно будет посмотреть измеряемые конфиги моделей. 

Обратите внимание на задание шейпов в конфиге.

Следующая команда запускает Model Analyzer с ручным конфигом. 

In [ ]:
!model-analyzer profile -f profile_config_manual.yaml

Если вы видите ошибку 
```
2021-12-11 12:42:17.115 ERROR[entrypoint.py:214] Model Analyzer encountered an error: Failed to set the value for field "triton_server_path". Error: Either the binary 'tritonserver' is not on the PATH, or Model Analyzer does not have permissions to execute os.stat on this path. 
```

Запустите
```
mkdir -p /opt/tritonserver
touch /workspace/install/bin/tritonserver
chmod u+x /workspace/install/bin/tritonserver
```
В версии 21.11 есть ошибка чрезмерной валидации входных параметров.

Если захочется приостановить выполнение команды, можно послать ей сигнал `SIGINT`. Это можно сделать либо нажав `Ctrl+C`, либо нажав на знак «стоп» рядом с ячейкой, либо (самое надёжное), выполнить в терминале внутри контейнера

```
kill -INT $(ps aux | grep model-ana | grep python | sed "s/^[[:alnum:]]*[[:space:]]*\([[:digit:]]*\).*/\1/")
```

При этом в логе должно появиться
```
INFO[analyzer_state_manager.py:174] Received SIGINT 1/3. Will attempt to exit after current measurement.
```

Это означает, что model-analyzer дождётся окончания текущего измерения и после этого сохранит текущий статус в checkpoint, из которого можно уже сделать предварительный анализ (см. дальше)

**После завершения профилировки** необходимо запустить анализ. Он выполнится очень быстро, никакой нагрузки на GPU он не создаёт.
Самые интересные результаты окажутся в папке results (сейчас там лежат пример результатов на NVIDIA A10). PDF-отчёт будет лежать в папке reports.

In [ ]:
!model-analyzer analyze -f profile_config_manual.yaml

Результаты будут выведены в терминал, общий анализ в PDF:

[reports/summaries/hifigan/result_summary.pdf](reports/summaries/hifigan/result_summary.pdf)

И подробности в csv файлы:

[results/metrics-model-gpu.csv](results/metrics-model-gpu.csv)

[results/metrics-model-inference.csv](results/metrics-model-inference.csv)

[results/metrics-server-only.csv](results/metrics-server-only.csv)

In [ ]:
# чтобы верифицировать yaml синтаксически
import yaml
with open("profile_config_manual.yaml") as pc:
    pc = yaml.safe_load(pc)

## Model Navigator

Model Navigator не установлен в контейнере с Triton SDK. Его контейнер необходимо собрать самостоятельно из исходников. [Инструкция.](https://github.com/triton-inference-server/model_navigator/blob/main/docs/quick_start.md#install-the-triton-model-navigator-and-run-container)

```bash
git clone https://github.com/triton-inference-server/model_navigator.git
# Optional
# git checkout v0.2.4
make docker
cd ..

docker run -it --rm \
 --gpus 1 \
 -v /var/run/docker.sock:/var/run/docker.sock \
 -v ${PWD}:${PWD} \
 -w ${PWD} \
 --net host \
 --name model-navigator \
 model-navigator /bin/bash
```

Конфигурация для Model Navigator хранится в файле [navigator_config.yaml](navigator_config.yaml). При работе Model Navigator склонен перезаписывать этот файл, поэтому команда запуска внутри контейнера предлагается такая:

```
cp navigator_config.yaml navigator_config_run.yaml; model-navigator run --config-path navigator_config_run.yaml
```